In [8]:
import pandas as pd
import os

# Define o caminho para o arquivo de dados
# A estrutura os.path.join é uma boa prática para criar caminhos de arquivos
# que funcionam em qualquer sistema operacional (Windows, Mac, Linux)
caminho_dados = os.path.join('..', 'data', 'MICRODADOS_ENEM_2023.csv')

# Tentativa de ler o arquivo com 'latin-1' encoding, comum em dados brasileiros
# Se ocorrer um erro, tente 'utf-8' ou 'iso-8859-1'
try:
    # Lendo o arquivo em chunks de 100.000 linhas
    iterator_chunks = pd.read_csv(
        caminho_dados,
        sep=';',
        encoding='latin-1',
        chunksize=100000
    )

    # Pegando apenas o primeiro chunk para análise inicial
    primeiro_chunk = next(iterator_chunks)

    print("Arquivo lido com sucesso!")

except Exception as e:
    print(f"Ocorreu um erro ao ler o arquivo: {e}")


# Se o chunk foi lido com sucesso, vamos inspecioná-lo
if 'primeiro_chunk' in locals():
    print("\nColunas do dataset:")
    # Imprime todas as colunas para termos uma visão geral
    print(primeiro_chunk.columns.tolist())

    print("\n\nPrimeiras 5 linhas do dataset:")
    display(primeiro_chunk.head())

Arquivo lido com sucesso!

Colunas do dataset:
['NU_INSCRICAO', 'NU_ANO', 'TP_FAIXA_ETARIA', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO', 'IN_TREINEIRO', 'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC', 'CO_UF_ESC', 'SG_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC', 'CO_MUNICIPIO_PROVA', 'NO_MUNICIPIO_PROVA', 'CO_UF_PROVA', 'SG_UF_PROVA', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC', 'TX_RESPOSTAS_MT', 'TP_LINGUA', 'TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC', 'TX_GABARITO_MT', 'TP_STATUS_REDACAO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006', 'Q007', 'Q008', 'Q009', 'Q010', '

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,210059085136,2023,14,M,2,1,1,1,17,1,...,C,C,B,B,A,B,B,A,A,B
1,210059527735,2023,12,M,2,1,0,1,16,1,...,B,A,B,B,A,A,C,A,D,B
2,210061103945,2023,6,F,1,1,1,1,0,1,...,B,A,A,B,A,A,A,A,A,B
3,210060214087,2023,2,F,1,3,1,2,0,2,...,A,A,A,B,A,A,D,A,A,B
4,210059980948,2023,3,F,1,3,1,2,0,2,...,A,A,A,B,A,A,B,A,A,A


In [9]:
import pandas as pd
import os
import time # Vamos importar a biblioteca time para cronometrar a execução

# 1. Definir as colunas que vamos usar
# Baseado na nossa investigação e no objetivo do projeto
colunas_de_interesse = [
    'NU_INSCRICAO',      # ID único do participante
    # --- Notas (nossas variáveis alvo e features importantes) ---
    'NU_NOTA_CN',        # Ciências da Natureza
    'NU_NOTA_CH',        # Ciências Humanas
    'NU_NOTA_LC',        # Linguagens e Códigos
    'NU_NOTA_MT',        # Matemática (NOSSO ALVO)
    'NU_NOTA_REDACAO',   # Redação
    # --- Questionário Socioeconômico ---
    'Q001', 'Q002',      # Escolaridade do pai e da mãe
    'Q006',              # Renda familiar
    'Q022',              # Celulares na residência
    'Q024',              # Computador na residência
    'Q025',              # Acesso à internet
    # --- Informações da Prova e Escola ---
    'TP_ESCOLA',         # Tipo de escola (pública, privada)
    'TP_LINGUA',         # Língua estrangeira escolhida
    'TP_STATUS_REDACAO', # Status da redação (sem problemas, anulada, etc)
    'TP_PRESENCA_CN',    # Presença na prova de CN
    'TP_PRESENCA_CH',    # Presença na prova de CH
    'TP_PRESENCA_LC',    # Presença na prova de LC
    'TP_PRESENCA_MT',    # Presença na prova de MT
]

# 2. Ler o arquivo em chunks, mas agora processando todos eles
caminho_dados = os.path.join('..', 'data', 'MICRODADOS_ENEM_2023.csv')

lista_de_chunks = []
total_linhas = 0

print("Iniciando a leitura otimizada do arquivo...")
start_time = time.time()

try:
    iterator_chunks = pd.read_csv(
        caminho_dados,
        sep=';',
        encoding='latin-1',
        chunksize=100000,
        usecols=colunas_de_interesse # <<< A MÁGICA ACONTECE AQUI!
    )
    
    for chunk in iterator_chunks:
        lista_de_chunks.append(chunk)
        total_linhas += len(chunk)
        print(f"Processado chunk. Total de linhas até agora: {total_linhas}")

    # 3. Concatenar todos os chunks em um único DataFrame
    df_enem_final = pd.concat(lista_de_chunks, ignore_index=True)
    
    end_time = time.time()
    print(f"\nLeitura completa! Tempo total: {end_time - start_time:.2f} segundos.")
    
    print("\nInformações do DataFrame final:")
    df_enem_final.info(memory_usage='deep')

except Exception as e:
    print(f"Ocorreu um erro: {e}")

Iniciando a leitura otimizada do arquivo...
Processado chunk. Total de linhas até agora: 100000
Processado chunk. Total de linhas até agora: 200000
Processado chunk. Total de linhas até agora: 300000
Processado chunk. Total de linhas até agora: 400000
Processado chunk. Total de linhas até agora: 500000
Processado chunk. Total de linhas até agora: 600000
Processado chunk. Total de linhas até agora: 700000
Processado chunk. Total de linhas até agora: 800000
Processado chunk. Total de linhas até agora: 900000
Processado chunk. Total de linhas até agora: 1000000
Processado chunk. Total de linhas até agora: 1100000
Processado chunk. Total de linhas até agora: 1200000
Processado chunk. Total de linhas até agora: 1300000
Processado chunk. Total de linhas até agora: 1400000
Processado chunk. Total de linhas até agora: 1500000
Processado chunk. Total de linhas até agora: 1600000
Processado chunk. Total de linhas até agora: 1700000
Processado chunk. Total de linhas até agora: 1800000
Processado 

In [10]:
import pandas as pd

# Supondo que 'df_enem_final' já está carregado na memória da célula anterior

# 1. Calcular a contagem de valores nulos por coluna
valores_nulos = df_enem_final.isnull().sum()

# 2. Calcular a porcentagem de valores nulos
porcentagem_nulos = (valores_nulos / len(df_enem_final)) * 100

# 3. Criar um DataFrame para visualizar o resumo
resumo_nulos = pd.DataFrame({
    'Contagem Nulos': valores_nulos,
    'Porcentagem Nulos (%)': porcentagem_nulos
})

# 4. Ordenar para ver as colunas mais problemáticas primeiro
resumo_nulos = resumo_nulos.sort_values(by='Porcentagem Nulos (%)', ascending=False)

print("Análise de Dados Faltantes:")
display(resumo_nulos)

Análise de Dados Faltantes:


,Contagem Nulos,Porcentagem Nulos (%)
NU_NOTA_CN,1241528,31.559283
NU_NOTA_MT,1241528,31.559283
NU_NOTA_CH,1111312,28.249230
NU_NOTA_REDACAO,1111312,28.249230
TP_STATUS_REDACAO,1111312,28.249230
NU_NOTA_LC,1111312,28.249230
NU_INSCRICAO,0,0.000000
TP_PRESENCA_CN,0,0.000000
TP_ESCOLA,0,0.000000
TP_PRESENCA_MT,0,0.000000


In [11]:
# Filtrando os dados com base na nossa estratégia

print(f"Tamanho original do DataFrame: {df_enem_final.shape}")

# Condições de filtro:
# 1. Presença em todas as quatro provas objetivas (código 1 = Presente)
filtro_presenca = (df_enem_final['TP_PRESENCA_CN'] == 1) & \
                  (df_enem_final['TP_PRESENCA_CH'] == 1) & \
                  (df_enem_final['TP_PRESENCA_LC'] == 1) & \
                  (df_enem_final['TP_PRESENCA_MT'] == 1)

# 2. Status da redação válido (código 1 = Sem problemas)
filtro_redacao = df_enem_final['TP_STATUS_REDACAO'] == 1

# Aplicando os filtros
df_enem_limpo = df_enem_final[filtro_presenca & filtro_redacao].copy() # .copy() evita o SettingWithCopyWarning

# Como boa prática, vamos remover as colunas de presença e status, pois elas agora são constantes (só têm valor 1)
colunas_para_remover = ['TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT', 'TP_STATUS_REDACAO']
df_enem_limpo.drop(columns=colunas_para_remover, inplace=True)


print(f"Tamanho do DataFrame após a limpeza: {df_enem_limpo.shape}")

# Verificação final: checar se ainda há nulos nas colunas de notas
print("\nVerificação de nulos no DataFrame limpo:")
display(df_enem_limpo[['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']].isnull().sum())

Tamanho original do DataFrame: (3933955, 19)
Tamanho do DataFrame após a limpeza: (2585115, 14)

Verificação de nulos no DataFrame limpo:


NU_NOTA_CN         0
NU_NOTA_CH         0
NU_NOTA_LC         0
NU_NOTA_MT         0
NU_NOTA_REDACAO    0
dtype: int64

In [13]:
# No final do notebook 01-exploracao-inicial.ipynb
import os

# Criar o caminho para a pasta de dados processados
caminho_processado = os.path.join('..', 'data', 'processed')
os.makedirs(caminho_processado, exist_ok=True)

# Salvar em formato Parquet, que é rápido e eficiente
df_enem_limpo.to_parquet(os.path.join(caminho_processado, 'enem_limpo.parquet'))
print("DataFrame limpo salvo com sucesso!")

DataFrame limpo salvo com sucesso!
